# PUMS Household Income vs. AMI (2021) in Phoenix  PUMAS

- https://www.census.gov/data/developers/data-sets/

For households by income and household size to compare to HUD AMI in same year
-  https://api.census.gov/data/2021/acs/acs1/pums/variables.html

2021 AMI by HH Size (from City of Phoenix)
- https://www.phoenix.gov/humanservicessite/Documents/2021%20AMI%20Limits%204.2.21.pdf

In [1]:
import pandas as pd
import math
import numpy as np
import os

In [2]:
import get_pums as get
import pums as calc
from AMI_BANDS_2021 import *

In [3]:
#Search parameters
y1 = '2021'

sample = 'acs1'

phx_pumas = ['0400113','0400114','0400115','0400116','0400117',\
             '0400118','0400119','0400120','0400121','0400122','0400123',\
             '0400125','0400128','0400112','0400129']

data_cols = 'SERIALNO,ST,PUMA,HINCP,NP,WGTP,ADJINC'

In [4]:
#assuming July 2021 $1 for HHInc equivalent to estimate 2023 Inc equivalent
inflation_adjust = 1.11

In [5]:
#Income to afford median household sales price by PUMA, conventional loan
inc_needed = {'0400112':176800,'0400113':131700,'0400114':89000,'0400115':82900,
              '0400116':87500,'0400117':115800,'0400118':82700,'0400119':79300,
              '0400120':109700,'0400121':84600,'0400122':61000,'0400123':68300,
              '0400125':73200,'0400128':90200,'0400129':126300}

In [6]:
#Income to afford median household sales price by PUMA, conventional loan
inc_needed_fha = {'0400112':195470,'0400113':145650,'0400114':98410,'0400115':91670,
              '0400116':96710,'0400117':128070,'0400118':91400,'0400119':87630,
              '0400120':121330,'0400121':93560,'0400122':67400,'0400123':75490,
              '0400125':80890,'0400128':99760,'0400129':139660}

In [7]:
# create a list of replicate weights
repwt = 'WGTP'
repwts = [repwt+str(i) for i in range(1, 81)]

## Get PUMA data

In [8]:
df = get.get_puma(sample,y1,data_cols)

In [9]:
df['GEO_ID'] = df['ST']+df['PUMA']
df = df[df.GEO_ID.isin(phx_pumas)]
df  = df.drop(['SERIALNO','ST','PUMA'],axis=1)
df = df[['GEO_ID']+[col for col in df.columns if col != 'GEO_ID']] #move id to first col
for col in df.columns[1:]: df[col] = df[col].astype(float)

In [10]:
df['HHSz'] = pd.cut(df['NP'],bins=[0,1,2,3,4,5,6,7,14],
                   labels=['1','2','3','4','5','6','7','8'])
df['HHSz'] = df['HHSz'].astype(str)

In [11]:
df['inc_needed'] = df['GEO_ID'].map(inc_needed)
df['inc_needed_fha'] = df['GEO_ID'].map(inc_needed_fha)

In [12]:
dff = df[~(df.HHSz.isna())&(df.HINCP!=-60000)&(df.HINCP!=0)].copy()
dff['HINCP'] = dff.ADJINC * dff.HINCP * inflation_adjust

In [13]:
dff['can_buy'] = np.where(dff.HINCP>=dff.inc_needed,'can buy','cannot afford')
dff['can_buy_fha'] = np.where(dff.HINCP>=dff.inc_needed_fha,'can buy fha','cannot afford fha')

In [14]:
#assign AMI range based on household size and ami dictionaries
dff['AMI_range'] = np.where((dff['HINCP']<=dff['HHSz'].map(AMI_30pct)),inc_lbl[0],
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_30pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_50pct)),inc_lbl[1],
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_50pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_80pct)),
                   inc_lbl[2],np.where((dff['HINCP']>dff['HHSz'].map(AMI_80pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_100pct)),
                   inc_lbl[3],np.where((dff['HINCP']>dff['HHSz'].map(AMI_100pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_120pct)),
                   inc_lbl[4],inc_lbl[5])))))

In [15]:
df_fha = dff.copy()

In [16]:
#make a column for each PUMA that assesses the number of households 
own_cols=[]
for k in inc_needed.keys():
    dff[f'aff_{k}_can']=np.where(dff.HINCP>=inc_needed[k],1,0)
    dff[f'aff_{k}_cant']=np.where(dff.HINCP<inc_needed[k],1,0)
    dff[f'aff_{k}_can_wt']=dff[f'aff_{k}_can']*dff.WGTP
    dff[f'aff_{k}_cant_wt']=dff[f'aff_{k}_cant']*dff.WGTP
    own_cols = own_cols + [f'aff_{k}_can',f'aff_{k}_cant'),
                          f'aff_{k}_can_wt',f'aff_{k}_cant_wt']

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (405126731.py, line 8)

In [ ]:
dff.head(3)

In [ ]:
#make a column for each PUMA that assesses the number of households 
own_cols_fha=[]
for k in inc_needed_fha.keys():
    df_fha[f'aff_{k}_fha_can']=np.where(df_fha.HINCP>=inc_needed_fha[k],1,0)
    df_fha[f'aff_{k}_fha_cant']=np.where(df_fha.HINCP<inc_needed_fha[k],1,0)
    df_fha[f'aff_{k}_fha_can_wt']=df_fha[f'aff_{k}_fha_can']*df_fha.WGTP
    df_fha[f'aff_{k}_fha_cant_wt']=df_fha[f'aff_{k}_fha_cant']*df_fha.WGTP
    own_cols_fha = own_cols_fha + [f'aff_{k}_fha_can',f'aff_{k}_fha_cant',
                                  f'aff_{k}_fha_can_wt',f'aff_{k}_fha_cant_wt']

In [ ]:
df_fha.head(3)

### table by PUMA for renters by AMI range - cost burdened vs. not cost burdened with conventional loan terms

In [ ]:
def make_est(df):
    df['hh_SE'] = df.apply(lambda x: (calc.get_se(x['WGTP'],x[repwts])),axis=1)
    df['hh_MOE'] = df.apply(lambda x: (calc.get_moe(x['hh_SE'])),axis=1)
    df['hh_CV'] = df.apply(lambda x: (calc.get_cv(x['WGTP'],x['hh_SE'])),axis=1)
    df.rename(columns={'WGTP':'hh'},inplace=True)
    return df

In [ ]:
drop_cols = ['HINCP','NP','ADJINC','HHSz','inc_needed','inc_needed_fha']

In [ ]:
table = dff.copy().drop(columns=drop_cols)

In [ ]:
buy_within = table.copy().drop(columns=own_cols).groupby(['GEO_ID','AMI_range','can_buy']).sum().reset_index()
buy_within = make_est(buy_within)
buy_within = buy_within.drop(columns=repwts)
buy_within = pd.pivot_table(buy_within,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',
                          columns=['AMI_range','can_buy'],aggfunc=np.sum).reset_index()

In [ ]:
buy_within.head(3)

In [ ]:
buy_all = table.copy().drop(columns=['AMI_range','can_buy']).groupby(['GEO_ID']).sum().reset_index()
buy_all = make_est(buy_all)
buy_all = buy_all.drop(columns=repwts)

In [ ]:
buy_all.head(3)

In [ ]:
fha = df_fha.copy().drop(columns=drop_cols)

In [ ]:
fha_within = fha.copy().drop(columns=own_cols_fha).groupby(['GEO_ID','AMI_range','can_buy_fha']).sum().reset_index()
fha_within = make_est(fha_within)
fha_within = fha_within.drop(columns=repwts)
fha_within = pd.pivot_table(fha_within,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',
                          columns=['AMI_range','can_buy_fha'],aggfunc=np.sum).reset_index()

In [ ]:
fha_within.head(3)

In [ ]:
fha_all = fha.copy().drop(columns=['AMI_range','can_buy_fha']).groupby(['GEO_ID']).sum().reset_index()
fha_all = make_est(fha_all)
fha_all = fha_all.drop(columns=repwts)

In [ ]:
fha_all.head(3)

In [ ]:
with pd.ExcelWriter(f'../../output/pums_gap_own/pums_own_gap_marketdata_120ami_inflation.xlsx') as writer:
    buy_within.to_excel(writer,sheet_name='can_cannot_withinpuma.xlsx')
    buy_all.to_excel(writer,sheet_name='can_cannot_atpuma.xlsx')
    fha_within.to_excel(writer,sheet_name='can_cannot_withinpuma_fha.xlsx')
    fha_all.to_excel(writer,sheet_name='can_cannot_atpuma_fha.xlsx')